In [0]:
select * from `workspace`.`default`.`car_sales` limit 100;
-------------------------------------------------------------------------------
--- Create processed table with typed and cleaned columns
SELECT
  vin AS saleid,
  lower(trim(make)) AS make,
  trim(model) AS model,
  try_cast(regexp_replace(year, '[^0-9]', '') AS INT) AS year_of_manufacture,
  NULL AS fuel_type,
  lower(trim(body)) AS body_type,
  coalesce(nullif(trim(state), ''), 'Unknown') AS region,
  NULL AS city,
  coalesce(nullif(trim(seller), ''), 'Unknown') AS dealership,
  try_to_timestamp(saledate, 'yyyy-MM-dd') AS sale_date,
  try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) AS selling_price,
  NULL AS cost_price,
  try_cast(regexp_replace(odometer, '[^0-9.]', '') AS DOUBLE) AS mileage,
  1 AS units_sold,
  (try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) * 1) AS total_revenue,
  CASE
    WHEN try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) IS NULL THEN NULL
    WHEN try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) = 0 THEN NULL
    ELSE (
      (try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) - NULL) / 
      try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)
    ) * 100
  END AS profit_margin_pct
FROM `workspace`.`default`.`car_sales`;

---------------------------------------------------------------------------------------------------------------------
---Adding performance tier and time columns (create view)
--CREATE OR REPLACE VIEW analytics.vw_car_sales_enriched AS
SELECT *,
  CASE
    WHEN profit_margin_pct IS NULL THEN 'Unknown'
    WHEN profit_margin_pct >= 20 THEN 'High'
    WHEN profit_margin_pct >= 10 THEN 'Medium'
    WHEN profit_margin_pct < 10 THEN 'Low'
    ELSE 'Unknown'
  END AS performance_tier,
  date_trunc('month', try_to_timestamp(saledate, 'yyyy-MM-dd')) AS sale_month,
  date_trunc('quarter', try_to_timestamp(saledate, 'yyyy-MM-dd')) AS sale_quarter,
  year(try_to_timestamp(saledate, 'yyyy-MM-dd')) AS sale_year
FROM (
  SELECT *,
    CASE
      WHEN try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) IS NULL THEN NULL
      WHEN try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) = 0 THEN NULL
      ELSE (
        (try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) - NULL) / 
        try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)
      ) * 100
    END AS profit_margin_pct
  FROM `workspace`.`default`.`car_sales`
)
;
------------------------------------------------------------------------------------------------------------------------
---Top 10 car makes by revenue
SELECT 
  lower(trim(make)) AS make,
  COUNT(1) AS units_sold,
  SUM(try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)) AS total_revenue,
  AVG(try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)) AS avg_selling_price,
  AVG(
    CASE
      WHEN try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) IS NULL THEN NULL
      WHEN try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) = 0 THEN NULL
      ELSE (
        (try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) - NULL) / 
        try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)
      ) * 100
    END
  ) AS avg_margin_pct
FROM `workspace`.`default`.`car_sales`
GROUP BY lower(trim(make))
ORDER BY total_revenue DESC
LIMIT 10;
------------------------------------------------------------------------------------------------------------------------
---Top 20 make/model pairs by revenue
SELECT 
  lower(trim(make)) AS make, 
  trim(model) AS model,
  COUNT(1) AS units_sold,
  SUM(try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)) AS total_revenue,
  AVG(try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)) AS avg_price
FROM `workspace`.`default`.`car_sales`
GROUP BY lower(trim(make)), trim(model)
ORDER BY total_revenue DESC
LIMIT 20;
------------------------------------------------------------------------------------------------------------------------
---Regional sales volumes (by city & region)
SELECT 
  coalesce(nullif(trim(state), ''), 'Unknown') AS region,
  NULL AS city,
  COUNT(1) AS units_sold,
  SUM(try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)) AS total_revenue,
  AVG(try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)) AS avg_price
FROM `workspace`.`default`.`car_sales`
GROUP BY coalesce(nullif(trim(state), ''), 'Unknown'), NULL
ORDER BY units_sold DESC
LIMIT 50;
------------------------------------------------------------------------------------------------------------------------
--- Correlation (price vs mileage)
SELECT
  corr(
    try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE),
    try_cast(regexp_replace(odometer, '[^0-9.]', '') AS DOUBLE)
  ) AS corr_price_mileage,
  corr(
    try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE),
    try_cast(regexp_replace(year, '[^0-9]', '') AS INT)
  ) AS corr_price_year,
  corr(
    try_cast(regexp_replace(odometer, '[^0-9.]', '') AS DOUBLE),
    try_cast(regexp_replace(year, '[^0-9]', '') AS INT)
  ) AS corr_mileage_year
FROM `workspace`.`default`.`car_sales`
WHERE
  try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) IS NOT NULL
  AND try_cast(regexp_replace(odometer, '[^0-9.]', '') AS DOUBLE) IS NOT NULL
  AND try_cast(regexp_replace(year, '[^0-9]', '') AS INT) IS NOT NULL;
------------------------------------------------------------------------------------------------------------------------
---Trend over time (monthly revenue & avg price)
SELECT 
  date_trunc('month', try_to_timestamp(saledate, 'yyyy-MM-dd')) AS sale_month,
  COUNT(1) AS units_sold,
  SUM(try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)) AS total_revenue,
  AVG(try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)) AS avg_selling_price
FROM `workspace`.`default`.`car_sales`
GROUP BY date_trunc('month', try_to_timestamp(saledate, 'yyyy-MM-dd'))
ORDER BY sale_month;
------------------------------------------------------------------------------------------------------------------------
---Profit margin distribution by performance tier
SELECT 
  CASE
    WHEN profit_margin_pct IS NULL THEN 'Unknown'
    WHEN profit_margin_pct >= 20 THEN 'High'
    WHEN profit_margin_pct >= 10 THEN 'Medium'
    WHEN profit_margin_pct < 10 THEN 'Low'
    ELSE 'Unknown'
  END AS performance_tier,
  COUNT(1) AS count_records,
  AVG(profit_margin_pct) AS avg_margin,
  SUM(total_revenue) AS sum_revenue
FROM (
  SELECT
    try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) AS selling_price,
    (try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) * 1) AS total_revenue,
    CASE
      WHEN try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) IS NULL THEN NULL
      WHEN try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE) = 0 THEN NULL
      ELSE NULL
    END AS profit_margin_pct
  FROM `workspace`.`default`.`car_sales`
) t
GROUP BY performance_tier
ORDER BY avg_margin DESC;

------------------------------------------------------------------------------------------------------------------------
---Emerging preference trends (fuel type over years)

SELECT 
  year(try_to_timestamp(saledate, 'yyyy-MM-dd')) AS sale_year, 
  NULL AS fuel_type,
  COUNT(1) AS units_sold,
  SUM(try_cast(regexp_replace(sellingprice, '[^0-9.]', '') AS DOUBLE)) AS revenue
FROM `workspace`.`default`.`car_sales`
GROUP BY year(try_to_timestamp(saledate, 'yyyy-MM-dd')), NULL
ORDER BY sale_year, units_sold DESC;
------------------------------------------------------------------------------------------------------------------------
--- Check duplicates by saleid (if available)

SELECT vin AS saleid, COUNT(1) AS ct
FROM `workspace`.`default`.`car_sales`
GROUP BY vin
HAVING COUNT(1) > 1
LIMIT 50;
------------------------------------------------------------------------------------------------------------------------
-- Null-check summary
SELECT
  SUM(CASE WHEN sellingprice IS NULL THEN 1 ELSE 0 END) AS num_null_selling_price,
  SUM(CASE WHEN saledate IS NULL THEN 1 ELSE 0 END) AS num_null_sale_date
FROM `workspace`.`default`.`car_sales`;
